This is the Notebook in which we will conduct the modeling. We have processed out data and done our feature engineering in another notebook. There we wrote that data to a csv file named car_details_cleaned_data.csv. We will conduct our modeling using that csv file and data. First we must import the nessesary libraries.

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

Next we will read in the data from the csv file and split the data into training and testing data.

In [17]:
# Load the dataset
car_df = pd.read_csv('car_details_cleaned_data.csv')

X = car_df.drop(columns=['selling_price'])  # Features
y = car_df['selling_price']  # Target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check dataset sizes
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3472, 41) (868, 41) (3472,) (868,)


So we see from the above, that our training set has 3472 datapoints and 41 features. Our training set has 868 datapoints and 41 features. Then we can see out targets are correct, having the same number of datapoints. This is perfect. Now we can attempt some modeling. I would like to begin with a nueral network approach; however, this network cannot be very deep as we have a huge number of attributes and not that large data so we must be very careful to prevent over training. 

In [18]:
# Define a regression neural network
"""model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input Layer
    Dense(32, activation='relu'),  # Hidden Layer
    Dense(16, activation='relu'),  # Hidden Layer
    Dense(1)  # Output Layer (1 neuron for continuous value)
])

# Compile the model (use mse for regression)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()"""

"model = Sequential([\n    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input Layer\n    Dense(32, activation='relu'),  # Hidden Layer\n    Dense(16, activation='relu'),  # Hidden Layer\n    Dense(1)  # Output Layer (1 neuron for continuous value)\n])\n\n# Compile the model (use mse for regression)\nmodel.compile(optimizer='adam', loss='mse', metrics=['mae'])\n\n# Summary of the model\nmodel.summary()"

Now the model is built, so we will next train the model.

In [19]:
# Train the model 
#history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [20]:
# Evaluate on test data
"""
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.2f}")  # Lower MAE is better
"""

'\ntest_loss, test_mae = model.evaluate(X_test, y_test)\nprint(f"Test MAE: {test_mae:.2f}")  # Lower MAE is better\n'

In [21]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) #standardization of features is often recomended for lasso models.

lasso_params = {'alpha': np.logspace(-3, 3, 10)}
lasso_grid = GridSearchCV(Lasso(), lasso_params, cv=5, scoring='r2')
lasso_grid.fit(X_train_scaled, y_train)
best_lasso_alpha = lasso_grid.best_params_['alpha']
# Lasso Regression

lasso_model = Lasso(alpha=best_lasso_alpha)

lasso_model.fit(X_train_scaled, y_train)

lasso_preds = lasso_model.predict(X_test_scaled) #produce predictions on the test data

print("MSE:", mean_squared_error(y_test, lasso_preds))
print("R² Score:", r2_score(y_test, lasso_preds)) #print results to see the accuracy

C:\Users\agjri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.121e+13, tolerance: 9.501e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\agjri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.038e+13, tolerance: 9.784e+10
  model = cd_fast.enet_coordinate_descent(
C:\Users\agjri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

MSE: 151222885845.28732
R² Score: 0.5044639247630169


In [ ]:
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted_Lasso': lasso_preds
})

# Save to CSV
results_df.to_csv('lasso_predictions.csv', index=False)

print("Predictions saved to lasso_predictions.csv")
